In [1]:
# Model Arguments 
from config import ModelArguments

config = ModelArguments()

# Multimodal Encoder, Resampler, Projector
from multimodal_encoder.builder import build_vision_tower
from multimodal_resampler.builder import build_vision_resampler
from multimodal_projector.builder import build_vision_projector

vision_tower = build_vision_tower(config)
vision_resampler = build_vision_resampler(config)
vision_projector = build_vision_projector(config)

On MacOS, decord package is not compatible with new python version.


/opt/homebrew/anaconda3/lib/python3.11/site-packages/torch/_utils.py:831: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  return self.fget.__get__(instance, owner)()


In [2]:
from llava_llama import LlavaLlamaForCausalLM # Register the llava models into 'transformers'
from transformers import AutoConfig
from transformers import AutoModelForCausalLM

config = AutoConfig.for_model("llava_llama", trust_remote_code=True)
model = AutoModelForCausalLM.from_config(config, trust_remote_code=True)

: 

In [2]:
# Dataset Construction for MultiModality

from dataprocess import LazySupervisedDataset
from config import DataArguments 
from llava_llama import prep_llava_llama_tokenizer

data_args = DataArguments(
    data_path = "data/mock.json",
    image_folder = "data/",
    video_folder = "data/",
    video_fps = 1,
    frames_upbound = 0,
    add_time_instruction = False,
    force_sample = False,
    default_fps = 10
)

tokenizer = prep_llava_llama_tokenizer("meta-llama/Meta-Llama-3.1-8B-Instruct")
dataset = LazySupervisedDataset(data_args=data_args, tokenizer=tokenizer, image_processor=vision_tower.image_processor)
data_dict = dataset[0]

tokenizer_config.json:   0%|          | 0.00/55.4k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.09M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/296 [00:00<?, ?B/s]

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [3]:
data_dict["input_ids"]

tensor([128000, 128006,   9125, 128007,    271,  38766,   1303,  33025,   2696,
            25,   6790,    220,   2366,     18,    198,  15724,   2696,     25,
           220,   1627,  10263,    220,   2366,     19,    271,   2675,    527,
           264,  11190,   4221,    323,  11376,  18328,     13,   1472,    527,
          3025,    311,   3619,    279,   9302,   2262,    430,    279,   1217,
          5825,     11,    323,   7945,    279,   1217,    449,    264,   8205,
           315,   9256,   1701,   5933,   4221,     13, 128009, 128006,   9125,
        128007,    271,  38766,   1303,  33025,   2696,     25,   6790,    220,
          2366,     18,    198,  15724,   2696,     25,    220,   1627,  10263,
           220,   2366,     19,    271, 128009, 128006,    882, 128007,    271,
          -200,    198,   3923,    649,    499,   1518,    304,    420,   2217,
            30, 128009, 128000, 128006,   9125, 128007,    271,  38766,   1303,
         33025,   2696,     25,   6790, 

In [ ]:
class LlavaLlamaTokenizer(LlamaTokenizer):
    def __init__(self, *args, **kwargs):
        super().__init__(*args, **kwargs)
        self.add_special_tokens({
            'pad_token': '[PAD]',
            'bos_token': '<|begin_of_text|>',
            'eos_token': '<|eot_id|>',
            'additional_special_tokens': ['<|start_header_id|>', '<|end_header_id|>']
        })

In [3]:
msg = [{"role": "user", "content": "hahaha"}, {"role": "assistant", "content": "hehehe"}]
tokenizer.apply_chat_template(msg, tokenize=False)

'<|begin_of_text|><|start_header_id|>user<|end_header_id|>\n\nhahaha<|eot_id|><|start_header_id|>assistant<|end_header_id|>\n\nhehehe<|eot_id|>'

In [5]:
special_tokens = ["<|begin_of_text|>", "<|start_header_id|>", "<|end_header_id|>", "<|eot_id|>"]
special_tokens_idx = [tokenizer.convert_tokens_to_ids(tok) for tok in special_tokens]
special_tokens_idx

# this is the standard manuver required for training llama class models ...
# tokenizer.add_special_tokens({'pad_token': '[PAD]'})
# model.resize_token_embeddings(len(tokenizer))

[128000, 128006, 128007, 128009]

In [8]:
# Right, now we try to propagate the data_dict into the model 
import torch 
from constants import IGNORE_INDEX

# # Assuming you have already loaded the model
# model = LlavaLlamaForCausalLM.from_pretrained("path_to_your_model")
# model.eval()  # Set the model to evaluation mode

# Prepare the inputs
input_ids = data_dict['input_ids'].unsqueeze(0)  # Add batch dimension
labels = data_dict['labels'].unsqueeze(0)  # Add batch dimension

# Prepare images, a list of per-patch-images tensors
images = []
if 'image' in data_dict:
    batch_images = [item[0] for item in data_dict["image"]]
    batch_images = torch.cat(batch_images, dim=0)
    images = [batch_images]

# Create attention mask
attention_mask = (labels != IGNORE_INDEX).long()

# Forward pass
# outputs = model(
#     input_ids=input_ids,
#     attention_mask=attention_mask,
#     labels=labels,
#     images=images,
# )

NameError: name 'model' is not defined

In [ ]:
# Error Tracing 
outputs = model(input_ids=input_ids, attention_mask=attention_mask, labels=labels, images=images) # Error occurs
model.prepare_inputs_labels_for_multimodal(input_ids, None, attention_mask, None, labels, images) # Same Error Propagated here

